# AI Config Create - Cookbook

This cookbook contains all code from `aiconfig-create/SKILL.md` for creating AI Configs.

## Prerequisites
- `LAUNCHDARKLY_API_TOKEN`: API token with `ai-configs:write` permission
- Project created (run `cookbook_aiconfig_projects.ipynb` first)
- Tools created (run `cookbook_aiconfig_tools.ipynb` first) - creates `search_knowledge_base` and `get_customer_info`

In [1]:
%pip install requests python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.

In [2]:
# Load environment variables from .env
import os
from dotenv import load_dotenv

def find_repo_root():
    """Find repo root by looking for .git directory"""
    path = os.getcwd()
    while path != "/":
        if os.path.exists(os.path.join(path, ".git")):
            return path
        path = os.path.dirname(path)
    return os.getcwd()

REPO_ROOT = find_repo_root()
ENV_FILE = os.path.join(REPO_ROOT, ".env")
load_dotenv(ENV_FILE)
print(f"Loaded environment from: {ENV_FILE}")

Loaded environment from: /Users/ld_scarlett/Documents/Github/agent-skills/.env


In [3]:
import requests
import os
import time

# Configuration
API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")
PROJECT_KEY = "support-ai"

def delete_config(config_key: str):
    """Delete an AI Config if it exists (for clean test runs)"""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}"
    headers = {"Authorization": API_TOKEN}
    response = requests.delete(url, headers=headers)
    if response.status_code == 204:
        print(f"[OK] Deleted existing config: {config_key}")
        return True
    elif response.status_code == 404:
        print(f"[INFO] Config '{config_key}' does not exist (clean slate)")
        return False
    else:
        print(f"[ERROR] Failed to delete config: {response.text}")
        return False

---
## Example 1: Complete Agent Mode Config (Two-Step Process)
From: `SKILL.md` lines 279-356

Creates an AI Config first, then adds a variation with proper model configuration.

**Important:** Tools cannot be attached via `defaultVariation` - they are ignored during config creation.
Use the two-step process: create config, then PATCH tools onto the variation.

**Note:** Tools (`search_knowledge_base`, `get_customer_info`) must be created first via `cookbook_aiconfig_tools.ipynb`

In [4]:
def create_agent_config():
    """
    Create an AI Config for agent mode using two-step process.
    
    Step 1: Create config without defaultVariation (for reliable model config)
    Step 2: Create variation with modelConfigKey and model configuration
    Step 3: PATCH tools onto the variation (tools are ignored in defaultVariation)
    """

    config_key = "support-agent"

    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json",
        "LD-API-Version": "beta"
    }

    # ============================================================
    # Step 1: Create AI Config (without defaultVariation)
    # ============================================================
    config_data = {
        "key": config_key,
        "name": "Customer Support Agent",
        "mode": "agent",
        "provider": {"name": "openai"},  # Provider at config level
        "modelName": "gpt-4o"             # Model at config level
    }

    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs"
    response = requests.post(url, headers=headers, json=config_data)

    if response.status_code not in [200, 201]:
        print(f"[ERROR] Failed to create config: {response.text}")
        return None

    print(f"[OK] Step 1: Created AI Config: {config_key}")
    time.sleep(0.5)

    # ============================================================
    # Step 2: Create variation with modelConfigKey
    # modelConfigKey is required for the model to display correctly in the UI
    # Format: {Provider}.{model-id}
    # ============================================================
    variation_data = {
        "key": "default",
        "name": "Default Configuration",
        "instructions": """You are a helpful customer support agent.

Your responsibilities:
- Answer customer questions
- Resolve issues efficiently
- Maintain a friendly tone

Company: {{company_name}}
Priority: {{support_priority}}""",
        "modelConfigKey": "OpenAI.gpt-4o",  # Required for UI display
        "model": {
            "modelName": "gpt-4o",
            "parameters": {
                "temperature": 0.7,
                "maxTokens": 2000,
                # Custom parameters for your application
                "custom_param": "value",
                "enable_feature": True,
                "workflow_config": {
                    "max_retries": 3,
                    "timeout": 30
                }
            }
        }
    }

    var_url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}/variations"
    var_response = requests.post(var_url, headers=headers, json=variation_data)

    if var_response.status_code not in [200, 201]:
        print(f"[ERROR] Failed to create variation: {var_response.text}")
        return None

    print(f"[OK] Step 2: Created variation 'default' with modelConfigKey OpenAI.gpt-4o")
    time.sleep(0.5)

    # ============================================================
    # Step 3: PATCH tools onto the variation
    # Tools cannot be attached via defaultVariation - must use PATCH
    # ============================================================
    patch_url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}/variations/default"
    tools_data = {
        "tools": [
            {"key": "search_knowledge_base", "version": 1},
            {"key": "get_customer_info", "version": 1}
        ]
    }
    
    patch_response = requests.patch(patch_url, headers=headers, json=tools_data)
    
    if patch_response.status_code == 200:
        print(f"[OK] Step 3: Attached 2 tools to variation 'default'")
    else:
        print(f"[WARNING] Could not attach tools: {patch_response.text}")

    print(f"\n[OK] Created AI Config: {config_key}")
    print(f"  URL: https://app.launchdarkly.com/projects/{PROJECT_KEY}/ai-configs/{config_key}")
    return var_response.json()

# Delete existing config first for clean test run, then create
print("=== Creating Agent Mode Config (Two-Step Process) ===\n")
delete_config("support-agent")
create_agent_config()

=== Creating Agent Mode Config (Two-Step Process) ===



[OK] Deleted existing config: support-agent


[OK] Step 1: Created AI Config: support-agent


[OK] Step 2: Created variation 'default' with modelConfigKey OpenAI.gpt-4o


[OK] Step 3: Attached 2 tools to variation 'default'

[OK] Created AI Config: support-agent
  URL: https://app.launchdarkly.com/projects/support-ai/ai-configs/support-agent


{'_id': '74e75277-e162-4155-bef3-a8984fe76632',
 '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs/support-agent',
   'type': 'application/json'}},
 '_publishedAt': 1770403680377,
 'createdAt': 1770403680377,
 'instructions': 'You are a helpful customer support agent.\n\nYour responsibilities:\n- Answer customer questions\n- Resolve issues efficiently\n- Maintain a friendly tone\n\nCompany: {{company_name}}\nPriority: {{support_priority}}',
 'key': 'default',
 'model': {'custom': {},
  'modelName': 'gpt-4o',
  'parameters': {'custom_param': 'value',
   'enable_feature': True,
   'maxTokens': 2000,
   'temperature': 0.7,
   'workflow_config': {'max_retries': 3, 'timeout': 30}}},
 'modelConfigKey': 'OpenAI.gpt-4o',
 'name': 'Default Configuration',
 'state': 'published',
 'version': 1}

---
## Example 2: Complete Completion Mode Config (Two-Step Process)
From: `SKILL.md` lines 358-437

Creates a completion mode config first, then adds a variation with modelConfigKey and tools.

**Note:** Uses same tools created by `cookbook_aiconfig_tools.ipynb`

In [5]:
def create_completion_config():
    """
    Create an AI Config for completion mode using two-step process.
    
    Step 1: Create config without defaultVariation
    Step 2: Create variation with modelConfigKey and messages
    Step 3: PATCH tools onto the variation
    """

    config_key = "content-assistant"

    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json",
        "LD-API-Version": "beta"
    }

    # ============================================================
    # Step 1: Create AI Config (without defaultVariation)
    # ============================================================
    config_data = {
        "key": config_key,
        "name": "Content Generation Assistant",
        "mode": "completion",
        "provider": {"name": "openai"},
        "modelName": "gpt-4o"
    }

    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs"
    response = requests.post(url, headers=headers, json=config_data)

    if response.status_code not in [200, 201]:
        print(f"[ERROR] Failed to create config: {response.text}")
        return None

    print(f"[OK] Step 1: Created AI Config: {config_key}")
    time.sleep(0.5)

    # ============================================================
    # Step 2: Create variation with modelConfigKey
    # ============================================================
    variation_data = {
        "key": "default",
        "name": "Default Configuration",
        "messages": [
            {
                "role": "system",
                "content": "You are a creative content writer for {{brand}}. Use available tools when needed."
            },
            {
                "role": "user",
                "content": "{{content_request}}"
            }
        ],
        "modelConfigKey": "OpenAI.gpt-4o",  # Required for UI display
        "model": {
            "modelName": "gpt-4o",
            "parameters": {
                "temperature": 0.9,
                "maxTokens": 2000,
                "topP": 0.95,
                # Custom parameters for your application
                "style_guide": "creative",
                "output_format": "markdown",
                "include_sources": True,
                "custom_settings": {
                    "tone": "friendly",
                    "audience": "general"
                }
            }
        }
    }

    var_url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}/variations"
    var_response = requests.post(var_url, headers=headers, json=variation_data)

    if var_response.status_code not in [200, 201]:
        print(f"[ERROR] Failed to create variation: {var_response.text}")
        return None

    print(f"[OK] Step 2: Created variation 'default' with modelConfigKey OpenAI.gpt-4o")
    time.sleep(0.5)

    # ============================================================
    # Step 3: PATCH tools onto the variation
    # ============================================================
    patch_url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-configs/{config_key}/variations/default"
    tools_data = {
        "tools": [
            {"key": "search_knowledge_base", "version": 1},
            {"key": "get_customer_info", "version": 1}
        ]
    }
    
    patch_response = requests.patch(patch_url, headers=headers, json=tools_data)
    
    if patch_response.status_code == 200:
        print(f"[OK] Step 3: Attached 2 tools to variation 'default'")
    else:
        print(f"[WARNING] Could not attach tools: {patch_response.text}")

    print(f"\n[OK] Created AI Config: {config_key}")
    print(f"  URL: https://app.launchdarkly.com/projects/{PROJECT_KEY}/ai-configs/{config_key}")
    return var_response.json()

# Delete existing config first for clean test run, then create
print("=== Creating Completion Mode Config (Two-Step Process) ===\n")
delete_config("content-assistant")
create_completion_config()

=== Creating Completion Mode Config (Two-Step Process) ===



[OK] Deleted existing config: content-assistant


[OK] Step 1: Created AI Config: content-assistant


[OK] Step 2: Created variation 'default' with modelConfigKey OpenAI.gpt-4o


[OK] Step 3: Attached 2 tools to variation 'default'

[OK] Created AI Config: content-assistant
  URL: https://app.launchdarkly.com/projects/support-ai/ai-configs/content-assistant


{'_id': '48f6f8cc-6b69-4ad2-82b3-efeff52e03b6',
 '_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-configs/content-assistant',
   'type': 'application/json'}},
 '_publishedAt': 1770403684150,
 'createdAt': 1770403684150,
 'key': 'default',
 'messages': [{'content': 'You are a creative content writer for {{brand}}. Use available tools when needed.',
   'role': 'system'},
  {'content': '{{content_request}}', 'role': 'user'}],
 'model': {'custom': {},
  'modelName': 'gpt-4o',
  'parameters': {'custom_settings': {'audience': 'general',
    'tone': 'friendly'},
   'include_sources': True,
   'maxTokens': 2000,
   'output_format': 'markdown',
   'style_guide': 'creative',
   'temperature': 0.9,
   'topP': 0.95}},
 'modelConfigKey': 'OpenAI.gpt-4o',
 'name': 'Default Configuration',
 'state': 'published',
 'version': 1}

---
## Summary

This cookbook created:
1. **support-agent** - Agent mode config with default variation
2. **content-assistant** - Completion mode config with default variation

Both configs include:
- `modelConfigKey` - Required for models to display correctly in UI (format: `{Provider}.{model-id}`)
- Tools (`search_knowledge_base`, `get_customer_info`) - attached via PATCH after variation creation
- Custom parameters in `model.parameters`
- Template variables (`{{variable}}`)

### Key Features Demonstrated
- **Two-step process** - Create config first, then add variation with `modelConfigKey`
- **PATCH tools separately** - Tools are ignored in `defaultVariation`, must be PATCHed after
- **modelConfigKey** - Required for UI display (e.g., `OpenAI.gpt-4o`, `Anthropic.claude-sonnet-4-5`)
- **Custom parameters** - Pass any custom data to your application via `model.parameters`

Next steps:
- Run `cookbook_aiconfig_variations.ipynb` to add more variations
- Run `cookbook_aiconfig_sdk.ipynb` to consume configs with the SDK